In [12]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image
import numpy as np

In [13]:
def embed_message(image_path, message, output_path):
    message += '###'  
    message_bits = ''.join(f'{ord(c):08b}' for c in message)
    
    image = Image.open(image_path)
    pixels = image.load()

    if len(message_bits) > image.width * image.height * 3:
        raise ValueError("Mesaj çok büyük, görüntüye sığmaz.")
    
    data_index = 0
    for y in range(image.height):
        for x in range(image.width):
            r, g, b = pixels[x, y]

            if data_index < len(message_bits):
                r = (r & 0xFE) | int(message_bits[data_index])
                data_index += 1
            if data_index < len(message_bits):
                g = (g & 0xFE) | int(message_bits[data_index])
                data_index += 1
            if data_index < len(message_bits):
                b = (b & 0xFE) | int(message_bits[data_index])
                data_index += 1

            pixels[x, y] = (r, g, b)

            if data_index >= len(message_bits):
                break

    image.save(output_path)

def extract_message(image_path):
    image = Image.open(image_path)
    pixels = image.load()
    message_bits = ''
    
    for y in range(image.height):
        for x in range(image.width):
            r, g, b = pixels[x, y]
            message_bits += str(r & 1)
            message_bits += str(g & 1)
            message_bits += str(b & 1)
    
    message = ''.join(chr(int(message_bits[i:i+8], 2)) for i in range(0, len(message_bits), 8))
    return message.split('###')[0]  # Sonlandırıcıyı temizle

def select_image():
    path = filedialog.askopenfilename(filetypes=[("PNG files", "*.png")])
    entry_image.delete(0, tk.END)
    entry_image.insert(0, path)

def save_image():
    path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png")])
    entry_output.delete(0, tk.END)
    entry_output.insert(0, path)

def hide_message():
    image_path = entry_image.get()
    output_path = entry_output.get()
    message = entry_message.get()
    if len(message) > 160:
        messagebox.showerror("Hata", "Mesaj 160 karakterden uzun olamaz.")
        return
    try:
        embed_message(image_path, message, output_path)
        messagebox.showinfo("Başarılı", "Mesaj başarıyla gizlendi.")
    except Exception as e:
        messagebox.showerror("Hata", str(e))

def reveal_message():
    image_path = entry_image.get()
    try:
        message = extract_message(image_path)
        messagebox.showinfo("Gizli Mesaj", message)
    except Exception as e:
        messagebox.showerror("Hata", str(e))


In [14]:
root = tk.Tk()
root.title("BPCS Steganografi")

tk.Label(root, text="Giriş Görüntüsü:").grid(row=0, column=0, sticky="e")
entry_image = tk.Entry(root, width=50)
entry_image.grid(row=0, column=1)
tk.Button(root, text="Seç", command=select_image).grid(row=0, column=2)

tk.Label(root, text="Çıkış Görüntüsü:").grid(row=1, column=0, sticky="e")
entry_output = tk.Entry(root, width=50)
entry_output.grid(row=1, column=1)
tk.Button(root, text="Kaydet", command=save_image).grid(row=1, column=2)

tk.Label(root, text="Mesaj:").grid(row=2, column=0, sticky="e")
entry_message = tk.Entry(root, width=50)
entry_message.grid(row=2, column=1, columnspan=2)

tk.Button(root, text="Mesajı Gizle", command=hide_message).grid(row=3, column=1)
tk.Button(root, text="Mesajı Göster", command=reveal_message).grid(row=3, column=2)

root.mainloop()